In [1]:
!pip install -q --upgrade transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
!pip uninstall pyarrow -y
!pip install -q pyarrow==12.0.0

Found existing installation: pyarrow 10.0.1
Uninstalling pyarrow-10.0.1:
  Successfully uninstalled pyarrow-10.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 24.9 MB/s eta 0:00:00


In [3]:
import pyarrow
print(pyarrow.__version__)

12.0.0


In [4]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00


In [5]:
import accelerate
print(accelerate.__version__)

0.27.2


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
ds = 'manual' # 'mrpc','manual'

input_path = '/content/drive/My Drive/Colab Notebooks/5_Corpora/corpora/'
pretrained_path = '/content/drive/My Drive/Colab Notebooks/8_Text_Paraphrasing/pretrained/parrot'
file_path = f'{ds}-triplet-corpus.csv'

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset
import torch
import pandas as pd

In [14]:
def preprocess_data(examples, tokenizer):
    inputs = ["paraphrase: " + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    labels = tokenizer(examples["target_text"], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
def fine_tune_parrot_model(dataset_path, output_dir):
    # Load the tokenizer and model specifically tailored for paraphrasing
    tokenizer = T5Tokenizer.from_pretrained('prithivida/parrot_paraphraser_on_T5')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = T5ForConditionalGeneration.from_pretrained('prithivida/parrot_paraphraser_on_T5')
    model.to(device)

    # Load the dataset
    df = pd.read_csv(dataset_path)
    dataset = df[['original', 'positive']].rename(columns={'original': 'input_text', 'positive': 'target_text'}) # Adjust based on sentiment
    dataset = Dataset.from_pandas(dataset)

    # Preprocess the dataset
    tokenized_dataset = dataset.map(lambda x: preprocess_data(x, tokenizer), batched=True)

    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        predict_with_generate=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,
        weight_decay=0.01,
    )

    # Initialize the trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        eval_dataset=tokenized_dataset, # Assuming you want to use the same dataset for simplicity; adjust as needed
        tokenizer=tokenizer,
    )

    # Train the model
    trainer.train()

    # Save the fine-tuned model and tokenizer
    # model.save_pretrained(output_dir)
    # tokenizer.save_pretrained(output_dir)

    return model, tokenizer

In [30]:
# Given that `prithivida/parrot_paraphraser_on_T5` is already fine-tuned for paraphrasing, you may not need to fine-tune further for similar tasks
# However, if you have a specific dataset or wish to further tailor the model, you can proceed with training as demonstrated earlier

model, tokenizer = fine_tune_parrot_model(folder_input_path + csv_file_path, folder_pretrained_path)

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.086402
2,No log,0.075281
3,No log,0.067671


Checkpoint destination directory /content/drive/My Drive/Colab Notebooks/8_Text_Paraphrasing/pretrained/parrot/checkpoint-53 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory /content/drive/My Drive/Colab Notebooks/8_Text_Paraphrasing/pretrained/parrot/checkpoint-106 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory /content/drive/My Drive/Colab Notebooks/8_Text_Paraphrasing/pretrained/parrot/checkpoint-159 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [ ]:
# Reminder: This example is structured for demonstrating how to load and potentially fine-tune a specific paraphrasing model
# For actual paraphrasing (inference), use the model's `generate` function with appropriate parameters

In [31]:
# Ensure model is in evaluation mode
# model.eval()

# Move model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

# Prepare your input text
input_text = "This is the sentence I want to paraphrase."
input_text = "paraphrase: " + input_text  # Add any necessary prefix
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
print(input_ids)

# Generate the paraphrase
outputs = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
print(outputs)

# Decode generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text: {generated_text}")


cuda
tensor([[ 3856, 27111,    10,   100,    19,     8,  7142,    27,   241,    12,
          3856, 27111,     5,     1]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], device='cuda:0')
Generated text: 


In [28]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# model_name = "t5-small"  # or any other T5 variant
model_name = "prithivida/parrot_paraphraser_on_T5"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

model.eval()
model.to(device)
print(device)

input_text = "paraphrase: This is the sentence I want to paraphrase."
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
print(input_ids)

attention_mask = torch.ones_like(input_ids)
outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=50)
# outputs = model.generate(input_ids, max_length=50)
# outputs = model.generate(input_ids, max_length=50, early_stopping=False, num_beams=1)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text with pretrained model: {generated_text}")


Generated text with pretrained model: This is the sentence I want to paraphrase.
